In [1]:
import polars as pl
from pyiceberg.catalog.rest import RestCatalog

pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)

polars.config.Config

# Querying the data

Now we have some data and we understand a bit better how Iceberg works under the hood. It's time to actually start using it!

The key selling point for Iceberg is that we have the option of using many different query engines to read from the same data storage.
To show this off, let's run some simple queries using a few different query engines. In this demo, we'll focus on locally runnable query engines, but often the pattern will be using something like Snowflake or Databricks for the heavy lifting, but have the optionality of using alternatives for different usecases.

First, we get a reference to our table, since many of these engines are using pyiceberg as a jumping-off point, either to directly interface with the Pyiceberg table, or because we can use Pyiceberg to find out the location of the current metadata.json file

In [2]:
catalog = RestCatalog(
    "lakekeeper", uri="http://lakekeeper:8181/catalog", warehouse="lakehouse"
)
table = catalog.load_table("housing.staging_prices")

## Pyiceberg

Let's see how we would use Pyiceberg directly to handle querying first. For each of these examples, we'll do something simple - we will calculate the mean monthly house price per month in 2024 (we've loaded data for 2024 and 2023 in the previous notebook). 

In [3]:
%%time

iceberg_results = table.scan(
    selected_fields=["price", "date_of_transfer"],
    row_filter="date_of_transfer >= '2024-01-01' and date_of_transfer <= '2024-12-31'",
)
iceberg_results.to_polars().group_by(pl.col("date_of_transfer").dt.month()).agg(
    pl.col("price").mean()
).sort(by="date_of_transfer")

CPU times: user 86.6 ms, sys: 40 ms, total: 127 ms
Wall time: 145 ms


date_of_transfer,price
i8,f64
1,"391,897.26"
2,"381,900.66"
3,"408,394.40"
4,"401,857.16"
5,"386,113.28"
…,…
8,"383,045.15"
9,"378,065.41"
10,"385,442.28"


## Polars
Pyiceberg provides us with limited filtering and projection capabilities - it provides the building blocks for libraries that build on top of Pyiceberg. We used Polars to finish the job in this example, but polars can read Iceberg directly so we can avoid the extra step

In [4]:
%%time
polars_df = (
    pl.scan_iceberg(table)
    .group_by(pl.col("date_of_transfer").dt.month())
    .agg(pl.col("price").mean())
    .sort(by="date_of_transfer")
    .collect()
)
polars_df

CPU times: user 111 ms, sys: 40 ms, total: 151 ms
Wall time: 120 ms


date_of_transfer,price
i8,f64
1,"394,187.58"
2,"384,490.04"
3,"409,735.65"
4,"401,145.10"
5,"395,103.79"
…,…
8,"396,430.32"
9,"390,961.38"
10,"389,016.05"


## Duckdb
Duckdb is also an excellent choice for working with Iceberg, especially if you want to stick to SQL.

It does require some setup, since Duckdb doesn't yet know how to talk to the REST catalog, so it needs to have it's own credentials, but the [duckdb-iceberg](https://github.com/duckdb/duckdb-iceberg) extension recently got additional sponsorship from AWS to improve Iceberg compatibility, so keep an eye on that

In [5]:
import duckdb

In [6]:
# Create a duckdb connection
conn = duckdb.connect()
# Load the Iceberg extension for DuckDB
conn.install_extension("iceberg")
conn.load_extension("iceberg")


# To be able to read the Iceberg metadata, we need credentials for the bucket
conn.sql("""
CREATE OR REPLACE SECRET minio (
TYPE S3,
ENDPOINT 'minio:9000',
KEY_ID 'minio',
SECRET 'minio1234',
USE_SSL false,
URL_STYLE 'path'
)
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [7]:
%%time
# We can read the iceberg data using DuckDB
conn.sql(f"""
SELECT month(date_of_transfer) as transfer_month, mean(price) as mean_price
FROM iceberg_scan('{table.metadata_location}')
GROUP BY 1
""").show()

┌────────────────┬────────────────────┐
│ transfer_month │     mean_price     │
│     int64      │       double       │
├────────────────┼────────────────────┤
│              1 │  394187.5805044869 │
│              2 │  384490.0360762256 │
│              3 │   409735.652211523 │
│              4 │  401145.1006167552 │
│              5 │   395103.789231015 │
│              6 │  402966.0933048433 │
│              7 │  396257.9457214263 │
│              8 │  396430.3194902848 │
│              9 │  390961.3840281215 │
│             10 │  389016.0510467688 │
│             11 │  372036.5243903345 │
│             12 │ 386641.94097710383 │
├────────────────┴────────────────────┤
│ 12 rows                   2 columns │
└─────────────────────────────────────┘

CPU times: user 237 ms, sys: 140 ms, total: 376 ms
Wall time: 676 ms


## Trino
Trino is another popular option, especially since AWS provides it as a serverless query engine through Athena. Trino is another SQL-based query engine, so the query looks pretty similar, just using Trino SQL dialect

In [8]:
import sqlalchemy as sa

engine = sa.create_engine("trino://trino:@trino:8080/lakekeeper")

sql = """
SELECT month(date_of_transfer) as transfer_month, avg(price) as mean_price 
FROM housing.staging_prices
GROUP BY 1
ORDER BY 1
"""

In [9]:
%%time
pl.read_database(sql, engine)

CPU times: user 269 ms, sys: 14.9 ms, total: 284 ms
Wall time: 4.08 s


transfer_month,mean_price
i64,f64
1,"394,187.58"
2,"384,490.04"
3,"409,735.65"
4,"401,145.10"
5,"395,103.79"
…,…
8,"396,430.32"
9,"390,961.38"
10,"389,016.05"


## Daft
Daft is a relatively new player in the Dataframe world, similar to Polars, but also designed for scaling out. It's also written in Rust, but Daft has had early support for Iceberg - let's see if that helps

In [10]:
import daft

In [11]:
%%time
(
    daft.read_iceberg(table)
    .groupby(daft.col("date_of_transfer").dt.month())
    .agg(daft.col("price").mean())
    .sort(by=daft.col("date_of_transfer"))
    .show(12)
)

date_of_transferUInt32,priceFloat64
1,394187.5805044869
2,384490.0360762256
3,409735.652211523
4,401145.1006167552
5,395103.789231015
...,...
8,396430.3194902848
9,390961.3840281215
10,389016.0510467688
11,372036.5243903345


CPU times: user 169 ms, sys: 63.9 ms, total: 233 ms
Wall time: 218 ms


## Query engines
So now we've done a tour of some of the query engines that are also easy to run locally - we've been through Python with Pyiceberg, Rust with Polars and Daft, C++ with Duckdb and finally Java with Trino. One important player we've left out here is Spark. There is no denying that Iceberg was originally a Java project and the Java Iceberg reference library is the most feature-complete. Pyiceberg is a close second though

![Iceberg Query Engines](images/iceberg_query_engine.svg)

In a real enterprise setup, you'll probably a managed service like Databricks or Snowflake that you can rely on as your main Iceberg driver - but the beauty of Iceberg is that you don't have to. You can mix and match these different query engines depending on the task at hand, while not having to move the data anywhere.

# Exercise

Try running a query using your favourite query engine to calculate the average house price for your county. If you don't live in the UK - pick the funniest sounding one. (I quite like WORCESTERSHIRE)

In [16]:
(
    pl.scan_iceberg(table)
    .filter(county="WORCESTERSHIRE")
    .select(pl.col("price").mean())
    .collect()
)

price
f64
"355,026.98"


In [18]:
# DuckDB
conn.sql(f"""
    SELECT mean(price) as mean_price
    FROM iceberg_scan('{table.metadata_location}')
    WHERE county = 'WORCESTERSHIRE'
""").show()

┌────────────────────┐
│     mean_price     │
│       double       │
├────────────────────┤
│ 355026.98243820295 │
└────────────────────┘

